# netkeiba URL取得

募集馬一覧CSVを読み込み、netkeibaの血統ページURLを自動取得してCSV出力する

In [1]:
import pandas as pd
from module.search import get_horse_url
from module.utils import normalize_name
import os
from tqdm import tqdm
import time

## 1. CSV読み込み処理

In [ ]:
# 入力CSVファイルを読み込み
input_file = 'target/target_list.csv'

try:
    df = pd.read_csv(input_file, encoding='utf-8')
    print(f"読み込み完了: {len(df)}頭の馬名")
    print(f"列名: {df.columns.tolist()}")
    print("\n最初の5頭:")
    print(df.head())
except Exception as e:
    print(f"CSVファイル読み込みエラー: {e}")
    df = None

## 2. URL取得処理

In [ ]:
if df is not None:
    # 結果を格納するリスト
    results = []
    
    # 各馬名に対してURL取得
    horse_column = df.columns[0]  # 最初の列を馬名列として使用
    
    print(f"\n{horse_column}列から馬名を取得してURL検索を開始...")
    
    for index, row in tqdm(df.iterrows(), total=len(df), desc="URL取得中"):
        
        horse_name = str(row[horse_column]).strip()
        
        if not horse_name or horse_name == 'nan':
            continue
            
        # 馬名を正規化
        normalized_name = normalize_name(horse_name)
        try:
            # URL取得
            url = get_horse_url(normalized_name)
            
            # 結果をリストに追加
            results.append({
                '馬名': normalized_name,
                'URL': url if url else 'URL取得失敗'
            })
            
            print(f"{index+1:3d}. {normalized_name:<30} → {'成功' if url else '失敗'}")
            
        except Exception as e:
            print(f"エラー: {normalized_name} - {e}")
            results.append({
                '馬名': normalized_name,
                'URL': f'エラー: {e}'
            })
        
        time.sleep(1)
    
    print(f"\nURL取得完了: {len(results)}件")

## 3. 結果確認

In [ ]:
if results:
    # 結果をDataFrameに変換
    result_df = pd.DataFrame(results)
    
    # 統計情報
    total_count = len(result_df)
    success_count = len(result_df[result_df['URL'].str.startswith('https://')])
    fail_count = total_count - success_count
    
    print(f"=== 取得結果サマリー ===")
    print(f"総数: {total_count}頭")
    print(f"成功: {success_count}頭 ({success_count/total_count*100:.1f}%)")
    print(f"失敗: {fail_count}頭 ({fail_count/total_count*100:.1f}%)")
    
    print("\n=== 最初の10件 ===")
    print(result_df.head(10))
    
    if fail_count > 0:
        print("\n=== 失敗した馬名 ===")
        failed_horses = result_df[~result_df['URL'].str.startswith('https://')]
        print(failed_horses)

## 4. CSV出力

In [ ]:
if results:
    # outputディレクトリが存在しない場合は作成
    os.makedirs('output', exist_ok=True)
    
    # 結果をCSVに保存
    output_file = 'output/horse_urls.csv'
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"\n結果を保存しました: {output_file}")
    print(f"ファイルサイズ: {os.path.getsize(output_file)} bytes")
    
    # 保存されたファイルの確認
    saved_df = pd.read_csv(output_file)
    print(f"保存確認: {len(saved_df)}行のデータが正常に保存されました")